# Submodule 4: Comparative genomics analysis
--------
## Overview
In this submodule, you will begin with a directory of proteomes from *de novo* assembled and annotated genomes. A single bacterial genome was analyzed in Submodules 1 & 2 and we automated the process on many genomes in Submodule 3 to produce a total of 10 genome sequences. We will add to this dataset with **reference genomes** publicly available from NCBI. These genome sequences are curated for quality and provide gene accessions with curated functional information. These datasets are crucial for providing context to our new dataset.

Genome assembly is a foundational step that allows researchers to generate a complete picture of the genetic material in a given organism. Assessing the quality and completeness of a genome assembly ensures that it accurately reflects the original genome, providing a reliable foundation for further analysis. **Comparative genomics** builds on genome assembly by comparing the genomes of different species or individuals. This field focuses on identifying similarities and differences in DNA sequences to understand evolutionary relationships and reveal the genetic underpinnings of adaptation and diversity. Comparative genomics can uncover how species evolve and adapt over time and identify genes associated with specific traits or diseases.

<p align="center">
  <img src="images/toxo_multi_alignment.png" width="80%"/>
</p>

### Learning Objectives

Through this submodule, users will gain experience in comparative genomics, resulting in an understanding of how to use existing genomes a reference points to better understand the context of a novel genome. You will learn to perform comparative genomic analyses to identify similarities and differences across genomes, run phylogenomic analyses, and construct pangenomes to capture genetic diversity. You will then apply these techniques to address biological questions and hypotheses.

- **Access Genome Datasets from the NCBI**:<br>
  Participants will use command line tools to acces genomes on NCBI for use in comparative genomics analyses.
    
- **Perform and Visualize Comparative Genomics Analyses**:<br>
  Gain proficiency in using comparative genomics tools and visuazing their outputs. Participants will use these outputs to identify patterns across genomes and develop hypotheses about genome relatedness, gene loss or gain events, and gene duplications.

- **Create and Understand Phylogeny Trees**:<br>
  Use both average nucleotide identity and ortholog groups in genes to create cladograms or phylogeny trees. Use phylogeny trees to gain an understanding of the species relatedness and to understand taxonomic groupings.

- **Draw Conclusions about Assembled Genome**:<br>
  Using the comparisons to fully annotated genomes, participants should be able to identify patterns in host and strain of comparator genomes and draw their own conclusions about the genomes assembled in submodules 1-3.

## **Install required software**

A few more tools are required for Submodule 4; OrthoFinder and fastANI are stand-alone software, while UpSet plots, seaborn, pandas, matplotlib, and BioPython are python modules. As with submodule 1, we will install these tools using __[Conda](https://docs.conda.io/en/latest/)__.

Each piece of software, along with links to publications and documentation, will be described in turn. Below is a brief summary of these tools.

### List of software
| **Tool**       | **Description**                                                                                                                                                           |
|:---------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **OrthoFinder**      | Finds orthogroups and orthologs, infers rooted gene trees for all orthogroups and identifies all of the gene duplication events in those gene trees.                         |
| **UpSet Plots**      | UpSet plots are a data visualization method for showing set data with more than three intersecting sets.                                                  |
| **fastANI**        | Developed for fast alignment-free computation of whole-genome Average Nucleotide Identity (ANI). ANI is defined as mean nucleotide identity of orthologous gene pairs shared between two microbial genomes.     
| **seaborn**        | A python library necessary for creating complex heatmaps.   |
| **pandas**        | A python library used for analyzing tabular data.   |
| **matplotlib**        | A python library used for generalized plotting purposes.   |
| **BioPython**        | A python library with a suite of tools used in bioinformatics.   |

In [ ]:
%%bash

# other installs are already complete, we just need to install seaborn
mamba install seaborn --quiet

In [ ]:
from IPython.display import Image

## Starting Data

This submodule begins with a directory of genomes in FASTA format and a directory of proteomes in FAA (FASTA amino acid) format. This module is designd to work with data produced from submodule 3, but feel free replace the FAA files within the directory *proteomes* or add additional FAA files as neeeded.

In [ ]:
%%bash

ls data/proteomes/

In [ ]:
%%bash

ls data/genomes/

## Process 1: Downloading Reference Genomes and Proteomes from NCBI
We have a directory of `genomes` and `proteomes` we created in submodules 1-3. To provide these context, we can also access thousands of deposited and annotated bacterial genomes and proteomes from NCBI with a few commands. For comparative genomics, we will grab several genomes in 

In [ ]:
%%bash

# download the list of all refseq assemblies
if [[ ! -s assembly_summary_refseq.txt ]]
then
    wget https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_refseq.txt -O assembly_summary_refseq.txt --quiet --no-check-certificate
fi

In [ ]:
%%bash

species="Campylobacter jejuni"

# we can use grep to search for our target organism
grep "$species" assembly_summary_refseq.txt | head -n 3

<div class="alert alert-block alert-info"><b>Tip</b>: Click on one of the blue highlighted links above to be taken to an assembly page on NCBIs server. Notice how the assembly page is laid out just like a directory and the link looks like a path. This is just like how our cloud server works!</div>

<p align="center">
  <img src="images/ncbi_screenshot.png" width="80%"/>
</p>

Let's use grep again to search for our organism, but this time we are going to `sort` the results randomly and take the top 10 results. We can then use `cut` to select only the 20th column which contains the link. We will also add an **outgroup** which should fall outside our other isolates in further analyses. After, we will loop through the links and download them by adding the file we want from the assembly directory to the end of the link.

In the case that multiple species are included in the analysis, separate species names can be searched with a pipe symbol (`|`).

In [ ]:
%%bash

# if you are running this analysis again, clean up the old files
if [[ -f .submodule4_run ]]
then
    rm query_list.txt reference_list.txt
    rm data/*/Campy*
    rm orthogroup_upsetplot.png
    rm -r orthofinder-analysis
    rm fastani_heatmap.png
    rm fastani_output.tsv
    rm .submodule4_run
fi

touch .submodule4_run

<div class="alert alert-block alert-info"><b>Tip</b>: Starting a file name with a <code>.</code> will typically hide it in any file browser or standard <code>ls</code>. Take a look at the difference between the two command outputs below.</div>

In [ ]:
%%bash

echo "=====Regular ls====="
ls

# -a stands for ALL
echo "=====All ls====="
ls -a

In [ ]:
%%bash

species="Campylobacter jejuni|Campylobacter coli"
outgroup="Campylobacter fetus"

links=$(grep -E "$species" assembly_summary_refseq.txt | sort -R | head -n 10 | cut -f 20)

# adding an outgroup for our analysis
# Add the link for the outgroup (just one for now)
outgroup_link=$(grep "$outgroup" assembly_summary_refseq.txt | sort -R | head -n 1 | cut -f 20)

# Append outgroup link to the species links
links="$links $outgroup_link"


for link in $links
do

    # gets the species corresponding to the isolate
    species=$(grep $link assembly_summary_refseq.txt | cut -f 8 | cut -f 1-2 -d' ' | sed 's/ /_/')
    # gets the accession number from the full link
    accession=$(echo $link | awk -F'/' '{print $NF}')
    
    echo Downloading $species from $(echo $link)

    # gets file names
    genome_file=${accession}_genomic.fna.gz
    proteome_file=${accession}_protein.faa.gz

    # gets fna download link
    fna=${link}/${genome_file}
    faa=${link}/${proteome_file}

    # wget downloads the file
    # -O specifies a name for the output file
    wget $fna -O data/genomes/${species}_${genome_file} --quiet --no-check-certificate
    wget $faa -O data/proteomes/${species}_${proteome_file} --quiet --no-check-certificate

done

In [ ]:
%%bash

# lastly, we have to unzip the genome and proteome files
gunzip data/genomes/*.gz
gunzip data/proteomes/*.gz


# echo list the two starting directories
echo '-----------------GENOMES--------------------'
ls data/genomes/
echo '-----------------PROTEOMES------------------'
ls data/proteomes/

We now have 11 genomes and 11 proteomes from NCBI which we can use for comparative genomics! Let's start by comparing our genomes to the NCBI isolates with FastANI.

## Process 2: Comparing Average Nucleotide Identity using FastANI
Program: FastANI - Fast alignment-free computation of whole-genome Average Nucleotide Identity (ANI)
Citation : Jain, C., Rodriguez-R, L.M., Phillippy, A.M. et al. *High throughput ANI analysis of 90K prokaryotic genomes reveals clear species boundaries.* Nat Commun 9, 5114 (2018). https://doi.org/10.1038/s41467-018-07641-9

Manual: https://github.com/ParBLiSS/FastANI

FastANI is developed for fast alignment-free computation of whole-genome Average Nucleotide Identity (ANI). ANI is defined as mean nucleotide identity of orthologous gene pairs shared between two microbial genomes. FastANI supports pairwise comparison of both complete and draft genome assemblies and avoids expensive sequence alignments in most ANI tools. With all our genomes on hand, we can make an initial comparison of ANI to give a preview of potential patterns among our genome set.

In [ ]:
%%bash

# take a look at the help menu
fastANI -h

We want to run FastANI with all of our genomes queried against each other. This corresponds to the example in the manual `fastANI --ql [QUERY_LIST] --rl [REFERENCE_LIST] -o [OUTPUT_FILE]`. The query and reference lists are lists of the paths to our genomes. Let's first make these files with some more bash looping.

In [ ]:
%%bash

# looping through all genome files in our genome directory
for genome in data/genomes/*
do
    # readlink gets the full path to the genome, tee writes the path to two files at once
    echo $genome | tee -a query_list.txt reference_list.txt
done

In [ ]:
%%bash

fastANI --ql query_list.txt --rl reference_list.txt -t 24 -o fastani_output.tsv --minFraction 0

In [ ]:
%%bash

# taking a look at our output
head -n 5 fastani_output.tsv

### Explanation of FastANI output

FastANI outputs a tab delimited file with a row for every query genome and five columns. The columns correspond to the query genome, the reference genome, ANI value, count of bidirectional fragment mappings, and total query fragments respectively. Genomes from the same species as our isolate should have an ANI >95%, while genomes from the same genus but different species will be lower.

For some downstream analysis, we should clean these names up a bit. We don't need the preceding data/genomes/ before every one of our genomes. Below, we use `sed` to edit a file in place, replacing all instance of `/data/genomes/` with empty text. More info on `sed` can be found in its manual page [here](https://linux.die.net/man/1/sed).

In [ ]:
%%bash

# sed syntax is 's|what we want to find|what we want to replace it with|'
# the -i flag specifies that the operation is in place and won't make a new file
# the g at the end of the sed expression stands for global--replacing every instance in the whole document
sed -i 's|data/genomes/||g' fastani_output.tsv

In [ ]:
%%bash

# we should see the effect of sed now
head -n 5 fastani_output.tsv

Starting from a clean FastANI output file, we can use some custom Python scripting to generate a heatmap and cladogram. Lets see how the scripts `fastANI_heatmap.py` runs first by calling the help menu.

In [5]:
%%bash

scripts/fastANI_heatmap.py -h

usage: fastANI_heatmap.py [-h] [--input INPUT] [--out OUT]

Creates a heatmap from fastANI results

options:
  -h, --help            show this help message and exit
  --input INPUT, -i INPUT
                        FastANI results to make a heatmap from
  --out OUT             Name of the resulting heatmap


The heatmap script just wants an input FastANI file in the `--input` flag, and an output file in the `--out` flag. Let's run this below:

In [ ]:
%%bash

scripts/fastANI_heatmap.py --input fastani_output.tsv --out fastani_heatmap.png

## A Briefer on Phylogeny

<p align="center">
  <img src="images/phylogeny_explanation.png" width="80%"/>
</p>

This image illustrates the concept of **phylogenetic trees**, which are used to represent evolutionary relationships among species. Each branch point, or node, represents a common ancestor shared by the species that diverge from it. The tree branches from left to right, moving from ancestors to present-day species. For example, the most recent common ancestor of species A and B is highlighted at the upper node connecting them. The equal branch lengths to A and B from their most recent common ancestor suggests that they have diverged equally over time. The root at the base of the tree represents the most recent common ancestor of all the species shown (A, B, C, D, and E). In this tree, species E is the **outgroup**, as it diverged earliest and shares a more distant common ancestor with the other species. Understanding these relationships helps scientists study how species have evolved over time and how they are related.

### Cladograms vs. Phylogenetic Trees
Both **cladograms** and **phylogenetic trees** are used to represent relationships among organisms, but they differ in the type of information they convey and how they are interpreted. A **cladogram** shows relationships based on shared derived characteristics without reflecting evolutionary time or genetic distance. Its branches are usually equal or arbitrary in length, focusing on the order of branching to indicate relative relationships and shared ancestry. In contrast, a **phylogenetic tree** provides a more detailed view of evolutionary history, incorporating both shared ancestry and the degree of divergence. Branch lengths in a **phylogenetic tree** represent *evolutionary time, genetic changes, or evolutionary distance*, depending on the data used. While a **cladogram** offers a qualitative view of relationships by highlighting who is more closely related to whom, a **phylogenetic tree** provides a quantitative perspective, often including a scale for time or genetic distance.

### Visualizing the ANI cladogram
Running the below cell with display the heatmap from FastANI which contains a cladogram. What do these results tell us? Is our outgroup the same species or different than the rest of our isolates? Do you see other interesting pattens?

In [ ]:
Image(filename='fastani_heatmap.png')

## Process 3: OrthoFinder Pairwise Orthology Analysis
Program: OrthoFinder: phylogenetic orthology inference for comparative genomics
Citation : Emms, D.M., Kelly, S. OrthoFinder: phylogenetic orthology inference for comparative genomics. Genome Biol 20, 238 (2019). https://doi.org/10.1186/s13059-019-1832-y

Manual: https://github.com/davidemms/OrthoFinder

There are many ways to make phylogenetic comparisons. FastANI, as it suggests in the name, is a fast option. However, Average Nucleotide Identity is not the most accurate predictor of evolutionary relationship. For this, we want to turn to ortholog analysis, that is, looking at the shared and unshared genes and their sequences among our isolates. We have a already added sequences from NCBI to our `proteomes` directory. Here, we will use them for comparative genomics analysis with **OrthoFinder**. The manual is very detailed, I recommend taking some time to read it.

The program takes a set of protein sequences for each isolate and runs pair-wise comarisons to identify orhtologous groups. For each orthogroup a gene tree is calculated and in the end an overall species tree is computed. To get a meaningful phylogenetic tree we need to be sure to include **at least four different genome datasets**. Ideally we would run this program with all of the avaialble sequences on NCBI. As you can imagine, a pairwise comparison with thousands of *Campylobacter* genomes would take a long time (days). We will therefore run it with our reduced set.

Start by counting the number of proteins in all the starting files. Think about what these numbers tell us right off the bat.

In [ ]:
%%bash

# grep -c will count the occurences of a searched string
grep -c '>' data/proteomes/*.faa

In [ ]:
%%bash

# as always, we should take a look at the help menu
orthofinder -h

We want to run Orthofinder with all of our proteomes queried against each other. For orthofinder, this simply requires pointing to the directory that our proteomes our in with `orthofinder -f <dir>`

<div class= "alert alert-block alert-info"><b>Tip</b>: Although there are no standards, in many programs <code>-o</code> is the the flag for an output file or directory, while <code>-t</code> typically corresponds to threads.</div></div>

In [ ]:
%%bash

# running orthofinder with thread and parallelization options to speed up analysis
orthofinder -t 24 -a 24 -f data/proteomes/ -o orthofinder-analysis

## Examine the output files

Let's take a look at the output directory from OrthoFinder first:

In [ ]:
%%bash

# orthofinder puts results in a directory titled Results_MonthDate
ls orthofinder-analysis/Results*/

We can see that OrthoFinder outputs a bunch of result files. Let's see what these files contain and highlight some of the important files.

### Orthogroups.tsv

A **tab** seperated table where each orthogroup is a row and each column is a different sample.

The table provides all of the data for orthogroups that are in at least two different samples. If a sample has more than one protein for that particular orthogroup than it will have a comma seperated list for the entry. Let's first take a look with `head` and then use some more advanced python methods with `pandas` to view the data table.

In [ ]:
%%bash

head -n 5 orthofinder-analysis/Results*/Orthogroups/Orthogroups.tsv

In [ ]:
# pandas is a library that deals with tables AKA dataframes
import pandas as pd

pd.read_csv("orthofinder-analysis/Results_*/Orthogroups/Orthogroups.tsv", 
            sep="\t")

Pandas is great not only for viewing the data, but also for performing operations on the data frame. Since many of the OrthoFinder files are tables, we will use pandas to inspect and analyze them.

### Orthogroups_UnassignedGenes.tsv

This table is in the same format as above, however this one contains Orthogroups that are unique to a single sample. As you scroll right you should notice that these Orthogroups are populated only in one sample each. 

In [ ]:
unassigned = pd.read_csv("orthofinder-analysis/Results_*/Orthogroups/Orthogroups_UnassignedGenes.tsv", sep="\t")
unassigned

We can use the `.count()` method on a `pandas` dataframe to count the number of non NaN values in each row. For the unassigned genes file, this corresponds to the number of unique Orthogroups per each sample. Run the below cell. Are the results what you expect? Does your outgroup contain the most unique Orthogroups? Do the number of unique Orthogroups correspond to the FastANI cladogram from above?

In [ ]:
unassigned.count()

### Orthogroups.GeneCount.tsv

This output file displays each orthogroup as a row while columns correspond to the number of genes (count) in that orthogroup per species. This can be easily parsed to see what orthogroups are specific to which species. Scrolling all the way to the right will show the total number of genes in each orthogroup.

In [ ]:
gene_counts = pd.read_csv("orthofinder-analysis/Results_*/Orthogroups/Orthogroups.GeneCount.tsv", sep="\t")
gene_counts

### SpeciesTree_rooted.txt

One of the most important outputs from OrthoFinder is the rooted-species tree. Unlike the fastANI cladogram above, this species tree is phylogenetically relevant, showing the evolutionary distance between the species in the analysis. This file is in *Newick* format, let's check out what that looks like.

In [ ]:
%%bash

cat orthofinder-analysis/Results_*/Species_Tree/SpeciesTree_rooted.txt

#### Newick Formatted Phylogeny Trees

This output seems a bit different from our example phylogenetic tree above. This is because the tree is in Newick format! The Newick format is a widely used way to represent phylogenetic trees in a compact, text-based manner. It encodes the tree structure using parentheses to group related taxa and commas to separate branches, with branch lengths optionally included as numerical values. Newick format is commonly used because it is machine-readable and compatible with various bioinformatics tools for analyzing and visualizing phylogenetic trees. Because bioinformatic file formats are typically uniform, we can use existing tools to convert our tree to a more human-friendly view.

Below, we will use `BioPython` to convert our text file to a 2-D phylogenetic tree.

In [ ]:
from Bio import Phylo
import matplotlib.pyplot as plt
import glob

# Here we use a wildcard to automatically expand the tree path
tree_path = glob.glob("orthofinder-analysis/Results_*/Species_Tree/SpeciesTree_rooted.txt")[0]

# Load the tree file into a Bio.Phylo object
tree = Phylo.read(tree_path, "newick", rooted=True)

# outgroup species separated by an underscore
outgroup_prefix = "Campylobacter_fetus"

# Find outgroup clade
outgroup = [clade for clade in tree.find_clades() if clade.name and clade.name.startswith(outgroup_prefix)][0]

# find the outgroup and reroot there
tree.root_with_outgroup(outgroup)
# add the location of the root node
tree.root.name = "Root"

fig, ax = plt.subplots(figsize=(10, 6))  # Create a figure and axes
Phylo.draw(tree, show_confidence=False, axes=ax)

#### Rooting Phylogeny Trees

We have mentioned the **outgroup** several times to this point, but what is its purpose? The **outgroup** is a species or clade that is closely related to the taxa of interest but not part of the group being studied. By including an **outgroup**, we can identify the most basal or ancestral traits in the tree, which help define the root and provide context for the branching patterns of the **ingroup**. Rooting a phylogenetic tree with an outgroup is essential for determining the evolutionary direction and establishing a point of reference for the tree.

For dissemenation of phylogenetic results, it is *essential* to leave in an outgroup used to root the tree. We will prune our outgroup here only as a way to to zoom in on our ingroup isolates.

In [ ]:
# Prune the specified outgroup node
tree.prune(outgroup)
tree.root.name = "Root"

fig, ax = plt.subplots(figsize=(10, 6))  # Create a figure and axes
Phylo.draw(tree, show_confidence=False, axes=ax)

## Interpreting Results in Context

Take a look at the phylogenetic tree we just made. Do you see these same patterns in the FastANI heatmap (provided in the cell below)? What do these results tell us? Do we see any intersting clades, or groupings of our isolates in context? If you are following along with the example data, you may notice that some of the SRR isolates cluster with *Campylobacter coli* and others cluster with *Campylobacter jejuni*. Do these align with our expectations?

In [ ]:
Image(filename='fastani_heatmap.png')

| **Isolate No.** | **SRA Accession** | **Species**  | **Resistance to Erythromycin (Macrolide)** | **Resistance to Ciprofloxacin (Fluoroquinolone)** | **Resistance Determinants (Ciprofloxacin)** | **Resistance to Tetracycline (Tetracycline)** | **Resistance Determinants (Tetracycline)** | **Resistance to Gentamicin (Aminoglycoside)** | **Resistance Determinants (Gentamicin)** | **Resistance to Streptomycin (Aminoglycoside)** | **Resistance Determinants (Streptomycin)** |
|------------------|-------------------|--------------|--------------------------------------------|--------------------------------------------------|---------------------------------------------|----------------------------------------------|---------------------------------------------|-----------------------------------------------|-------------------------------------------|------------------------------------------------|---------------------------------------------|
| 72               | SRR10067958      | *C. jejuni*  | S                                          | S                                                | —                                           | R                                            | —                                           | S                                             | —                                         | S                                              | —                                           |
| 91               | SRR10068079      | *C. jejuni*  | S                                          | S                                                | gyrA_CJ[86:T-I]                             | S                                            | tet(O)                                      | S                                             | —                                         | S                                              | —                                           |
| 145              | SRR10068117      | *C. jejuni*  | S                                          | S                                                | —                                           | R                                            | —                                           | S                                             | —                                         | S                                              | —                                           |
| 193              | SRR10056681      | *C. jejuni*  | S                                          | S                                                | gyrA_CJ[86:T-I]                             | R                                            | tet(O)_2                                    | S                                             | —                                         | R                                              | —                                           |
| 213              | SRR10056829      | *C. jejuni*  | S                                          | S                                                | —                                           | S                                            | tet(O)                                      | S                                             | —                                         | S                                              | ant(6)-Ia, aadE-Cp2                        |
| 214              | SRR10056914      | *C. coli*    | S                                          | S                                                | —                                           | S                                            | —                                           | S                                             | —                                         | S                                              | —                                           |
| 242              | SRR10056778      | *C. coli*    | S                                          | S                                                | —                                           | S                                            | —                                           | S                                             | —                                         | R                                              | —                                           |
| 248              | SRR10056855      | *C. jejuni*  | S                                          | S                                                | 23s_CJ[2074:A-M], gyrA_CJ[86:T-I; 90:D-N]  | R                                            | tet(O)_2                                    | S                                             | —                                         | S                                              | —                                           |
| 263              | SRR10056784      | *C. jejuni*  | S                                          | S                                                | gyrA_CJ[86:T-I]                             | R                                            | tet(O)_2                                    | S                                             | —                                         | R                                              | ant(6)-Ia, aadE-Cp2                        |
| 276              | SRR10056856      | *C. jejuni*  | S                                          | S                                                | —                                           | S                                            | —                                           | S                                             | —                                         | R                                              | —                                           |



In [ ]:
%%bash

scripts/upset_plot_from_bins_tsv.py --help

In [ ]:
%%bash

scripts/upset_plot_from_bins_tsv.py --genes orthofinder-analysis/Results_*/Orthogroups/Orthogroups.GeneCount.tsv --out orthogroup_upsetplot.png

In [ ]:
Image('orthogroup_upsetplot.png')

## Conclusion

End of submodule 4. After learning about how genome sequencing works (and where it can go wrong) we went through standard processing steps (FASTQC and FASTP) and ultimetly ran a genome assembly tool. The final output from spades represents our draft genome sequence.

Be sure to shut down your instance. We will run one more bit of code to copy our genome file for use in submodule 2.